# Instruction tuning - Supervised finetuning

Pre-training = reading a textbook
Supervised finetuning = Imitating

- SFT helps the model how the knowledge learnt is applied.
- SFT does not teach new facts. It teached new behaviors.
- SFT teached models to shift the model's probability distributions to favor the types of responses in the training dataset
- The model mimics the patterns in the training data. So, invetime time in data curation
- Quality of training data matters in SFT. In pretraining, quantity matters.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import load_dataset
import trackio as wandb
from trl import SFTTrainer, SFTConfig

/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(project="my-sft", name="sft-runs")

* Trackio project initialized: my-sft
* Trackio metrics logged to: /Users/phani/.cache/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "my-sft"
* or by running in Python: trackio.show(project="my-sft")


Traceback (most recent call last):
  File "/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/gradio/blocks.py", line 2246, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/gradio/blocks.py", line 1908, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^

In [4]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

In [6]:
data = load_dataset("roneneldan/TinyStories")
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [7]:
data['train']

Dataset({
    features: ['text'],
    num_rows: 2119719
})

In [20]:
config = SFTConfig(
    output_dir="./models",
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    max_steps=20,
    logging_steps=2,
    report_to="trackio"
)

In [21]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'].select(range(100)),
    args=config
)

In [22]:
trainer.train()

/Users/phani/Work/Adhoc/projects/relearning-ai/learn-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
2,1.188000
4,1.008800
6,1.337200
8,1.388900
10,1.646300
12,1.808200
14,1.827100
16,1.884600
18,2.395800
20,2.358500


* Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)


TrainOutput(global_step=20, training_loss=1.684325921535492, metrics={'train_runtime': 31.3744, 'train_samples_per_second': 1.275, 'train_steps_per_second': 0.637, 'total_flos': 21338005438464.0, 'train_loss': 1.684325921535492, 'epoch': 0.4})